In [1]:
import osmnx as ox
import geopandas as gpd
import os
import datetime
import re
import requests
import numpy as np
import arcpy
from arcpy import env
#from osmnx.geometries import geometries_from_place


In [2]:
# Function to clean table names
def func_clean_table_name(filename):
    # Replace forbidden characters with underscores
    cleaned_name = re.sub(r'[^a-zA-Z0-9_]', '_', os.path.splitext(filename)[0])
    # Ensure the name doesn't start with a number
    if cleaned_name[0].isdigit():
        cleaned_name = f"_{cleaned_name}"
    return cleaned_name

In [3]:
#pull_location = "Hawaii County, Hawaii, USA"
pull_location = "Honolulu County, Hawaii, USA"
clean_loc_name = func_clean_table_name(f"{pull_location}")
print(f"Clean Name from Location: {clean_loc_name}")

Clean Name from Location: Honolulu_County__Hawaii__USA


In [4]:
current_dir = os.getcwd()
print(current_dir)

C:\jup\buildings_osm_extract


In [5]:
# Create an output subfolder in the same folder as this notebook file, named for current date and time
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder_name = f"output\output_{current_time}"

# Construct the full path for the new output folder
output_folder_path = os.path.join(current_dir, output_folder_name)

# Create the folder
os.makedirs(output_folder_path, exist_ok=True)

destination_dir = output_folder_path # This is so the destination can be changed easily without messing up the previous code

print(f"Location of this notebook: {current_dir}")
print(f"Destination folder: {destination_dir}")

Location of this notebook: C:\jup\buildings_osm_extract
Destination folder: C:\jup\buildings_osm_extract\output\output_20250414_192121


In [6]:
geopackage_name = clean_loc_name + ".gpkg"
geopackage_path = os.path.join(destination_dir, geopackage_name)
print(f"Destination GPKG: {geopackage_path}")

Destination GPKG: C:\jup\buildings_osm_extract\output\output_20250414_192121\Honolulu_County__Hawaii__USA.gpkg


In [7]:
# Define the place to pull buildings for
mytags = {"building": True}
gdf_temp = ox.features_from_place(pull_location, tags=mytags)
print(gdf_temp.geom_type.value_counts())

C:\Users\JPL\AppData\Roaming\Python\Python311\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 106 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Polygon         68755
Point              43
MultiPolygon       22
Name: count, dtype: int64


In [8]:
#Filter to polygon part of data only
gdf_polygons = gdf_temp[gdf_temp.geom_type.isin(["Polygon", "MultiPolygon"])]
print("Data filtered to polygons only")

Data filtered to polygons only


In [9]:
print(gdf_polygons.crs)

epsg:4326


In [10]:
gdf_polygons.to_file(geopackage_path, layer=clean_loc_name, driver="GPKG")
print(f"Geopackage at {geopackage_path} now contains building footprints in layer {clean_loc_name}")

Geopackage at C:\jup\buildings_osm_extract\output\output_20250414_192121\Honolulu_County__Hawaii__USA.gpkg now contains building footprints in layer Honolulu_County__Hawaii__USA


In [11]:
# Export to Shapefile
#gdf_polygons.to_file(shapefile_path)
#print(f"Shapefile created at: {shapefile_path}")